Based on these posts: 
* https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
* https://github.com/keisukeirie/Amazon_review_helpfulness_prediction
* https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
* https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

In [ ]:
!pip install -q boto3
!pip install -q xgboost

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
!head ./amazon20000.csv

In [ ]:
df = pd.read_csv('amazon20000.csv')
df.shape

In [ ]:
df.head(5)

# Drop any NaNs

In [ ]:
df.shape

In [ ]:
df.isna().values.any()

In [ ]:
df_cleaned_and_filtered = df.dropna()
df_cleaned_and_filtered = df_cleaned_and_filtered.reset_index()
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered = df_dropna.query('helpful_votes > 0')
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['pct_helpful_votes'] = df_cleaned_and_filtered['helpful_votes'] / df_cleaned_and_filtered['total_votes']
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['is_helpful'] = df_cleaned_and_filtered['pct_helpful_votes'] > 0.75
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['is_positive_sentiment'] = (df_cleaned_and_filtered['star_rating'] >= 4).astype(int)
df_cleaned_and_filtered.head(5)

In [ ]:
import seaborn as sns

sns.countplot(x='is_positive_sentiment', data=df_cleaned_and_filtered)


## Balance the dataset

In [ ]:
from sklearn.utils import resample

is_negative_sentiment_df = df_cleaned_and_filtered.query('is_positive_sentiment == 0')
is_positive_sentiment_df = df_cleaned_and_filtered.query('is_positive_sentiment == 1')

is_positive_downsampled_df = resample(is_positive_sentiment_df,
                                      replace = False, # sample without replacement
                                      n_samples = len(is_negative_sentiment_df), # match minority n
                                      random_state = 27) # reproducible results

df_cleaned_and_filtered = pd.concat([is_negative_sentiment_df, is_positive_downsampled_df])


In [ ]:
sns.countplot(x='is_positive_sentiment', data=df_cleaned_and_filtered)


Create X (features) and y (labels)

In [ ]:
X = df_cleaned_and_filtered[['review_body']]
y = df_cleaned_and_filtered['is_positive_sentiment']

print('X.shape:  {}'.format(X.shape))
print('y.shape:  {}'.format(y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [ ]:
X_train.head(5)

In [ ]:
y_train.head(5)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
    
#class NumberSelector(BaseEstimator, TransformerMixin):
#    def __init__(self, field):
#        self.field = field
#    def fit(self, X, y=None):
#        return self
#    def transform(self, X):
#        return X[[self.field]]

In [ ]:
import nltk
import re
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

### Create Scikit-Learn Pipeline with SGDClassifier

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
#from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier

classifier = Pipeline([
    ('features', FeatureUnion([
        ('body', Pipeline([
            ('body_text_selector', TextSelector('review_body')),
            ('tfidf_vectorizer', TfidfVectorizer(tokenizer=Tokenizer, stop_words="english",
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
#        ('star_rating', Pipeline([
#            ('star_rating_selector', NumberSelector('star_rating')),
#            ('standard_scaler', StandardScaler()),
#        ])),
    ])),
    ('classifier', SGDClassifier(learning_rate='optimal'))
#    ('classifier', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('classifier', RandomForestClassifier()),
    ])

Fit the model

In [ ]:
classifier.fit(X_train, y_train)

Predict and calculate metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

preds = classifier.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, preds))
print('Precision: ', precision_score(y_test, preds, average=None))

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
df_cm = confusion_matrix(y_test, preds)
df_cm

In [ ]:
# TODO:  Explain the classifier

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#plt.figure(figsize = (10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

# TODO:  Balance the dataset before training
# TODO:  Add labels to each quadrant (predicted y-axis, actual x-axis)

plt.show()

### Create Scikit-Learn Pipeline with XGBoostClassifier

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
#from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#from sklearn.linear_model import SGDClassifier

classifier = Pipeline([
    ('features', FeatureUnion([
        ('body', Pipeline([
            ('body_text_selector', TextSelector('review_body')),
            ('tfidf_vectorizer', TfidfVectorizer(tokenizer=Tokenizer, stop_words="english",
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
#        ('star_rating', Pipeline([
#            ('star_rating_selector', NumberSelector('star_rating')),
#            ('standard_scaler', StandardScaler()),
#        ])),
    ])),
#    ('classifier', SGDClassifier(learning_rate='optimal'))
    ('classifier', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('classifier', RandomForestClassifier()),
    ])

Fit the model

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

preds = classifier.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, preds))
print('Precision: ', precision_score(y_test, preds, average=None))

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
df_cm = confusion_matrix(y_test, preds)
df_cm

In [ ]:
# TODO:  Explain the classifier

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#plt.figure(figsize = (10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

# TODO:  Add labels to each quadrant (False, True / False, True)

plt.show()

### BERT
MultiLabel Classification:  https://towardsdatascience.com/multi-label-classification-using-bert-roberta-xlnet-xlm-and-distilbert-with-simple-transformers-b3e0cda12ce5

BinaryClassification:  https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3

Install simpletransformers by Hugging Face folks

In [ ]:
!pip install simpletransformers==0.19.9

In [ ]:
!pip install tensorboardx==2.0

In [ ]:
!pip install torch==1.4.0

### Prepare the dataset for BERT.

By `simpletransformer` convention, the dataframe must have 2 columns:
* `text`
* `labels`

In [ ]:
df_bert = df_cleaned_and_filtered[['review_body', 'is_positive_sentiment']]
df_bert.columns = ['text', 'labels']
df_bert.head(5)

### Split the data into train and test.  

In [ ]:
from sklearn.model_selection import train_test_split

df_bert_train, df_bert_test = train_test_split(df_bert, test_size=0.10)

### Train the model

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a TransformerModel

args = {
#   'model_type':  'bert',
#   'model_name': 'bert-base-cased',
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': False,
#   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 1,
   'weight_decay': 0,
   'learning_rate': 3e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'tensorboard_dir': 'runs',
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

bert_model = ClassificationModel(model_type='bert', #roberta, etc.
                                 model_name='bert-base-cased',
                                 args=args,
                                 use_cuda=False)

bert_model.train_model(train_df=df_bert_train,
                       eval_df=df_bert_test,
                       show_running_loss=True)

In [ ]:
import sklearn

result, model_outputs, wrong_predictions = bert_model.eval_model(eval_df=df_bert_test, acc=sklearn.metrics.accuracy_score)

In [ ]:
predictions, raw_outputs = bert_model.predict(["""Very funny. A typical mid 50's comedy."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = bert_model.predict(["""That movie was absolutely awful."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))